In [5]:
from projectq import MainEngine  # import the main compiler engine
from projectq.ops import All, CNOT, NOT, H, Toffoli, Measure  # import the operations we want to perform 
from projectq.backends import CircuitDrawer

In [6]:
sumando_1 = input("Primer sumando en binario (4 bits)")
sumando_2 = input("Segundo sumando en binario(4 bits)")

Primer sumando en binario (4 bits)0001
Segundo sumando en binario(4 bits)0001


In [15]:
n = 4

backend = CircuitDrawer()
#ResourceCounter()
eng = MainEngine(backend)
#eng = MainEngine()

a = eng.allocate_qureg(n)
b = eng.allocate_qureg(n+1)
c = eng.allocate_qureg(n)

for i in range(n):
    if sumando_1[i] == "1":
        NOT | (a[n - (i+1)])
for i in range(n):
    if sumando_2[i] == "1":
        NOT | (b[n - (i+1)])
        
for i in range(n-1):
    Toffoli | (a[i], b[i], c[i+1])
    CNOT | (a[i], b[i])
    Toffoli | (a[i], b[i], c[i+1])

Toffoli | (a[n-1], b[n-1], b[n])
CNOT | (a[n-1], b[n-1])
Toffoli | (a[n-1], b[n-1], b[n])  

CNOT | (c[n-1], b[n-1])

for i in range(n-1):
    Toffoli | (c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    CNOT | (a[(n-2)-i], b[(n-2)-i])
    Toffoli | (a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    
    CNOT | (c[(n-2)-i], b[(n-2)-i])
    CNOT | (a[(n-2)-i], b[(n-2)-i])

All(Measure) | b

eng.flush()

result = []
for i in range(n):
    result.append(int(b[i]))

#my_backend = Aer.get_backend("qasm_simulator")  
#job = execute(qc, my_backend, shots=20)
#job_stats = job.result().get_counts()
print(result)
#print(qc)
#print(backend.get_latex()) 
    
    

[0, 0, 0, 0]


In [19]:

backend.get_latex()


'\\documentclass{standalone}\n\\usepackage[margin=1in]{geometry}\n\\usepackage[hang,small,bf]{caption}\n\\usepackage{tikz}\n\\usepackage{braket}\n\\usetikzlibrary{backgrounds,shadows.blur,fit,decorations.pathreplacing,shapes}\n\n\\begin{document}\n\\begin{tikzpicture}[scale=0.8, transform shape]\n\n\\tikzstyle{basicshadow}=[blur shadow={shadow blur steps=8, shadow xshift=0.7pt, shadow yshift=-0.7pt, shadow scale=1.02}]\\tikzstyle{basic}=[draw,fill=white,basicshadow]\n\\tikzstyle{operator}=[basic,minimum size=1.5em]\n\\tikzstyle{phase}=[fill=black,shape=circle,minimum size=0.1cm,inner sep=0pt,outer sep=0pt,draw=black]\n\\tikzstyle{none}=[inner sep=0pt,outer sep=-.5pt,minimum height=0.5cm+1pt]\n\\tikzstyle{measure}=[operator,inner sep=0pt,minimum height=0.5cm, minimum width=0.75cm]\n\\tikzstyle{xstyle}=[circle,basic,minimum height=0.35cm,minimum width=0.35cm,inner sep=-1pt,very thin]\n\\tikzset{\nshadowed/.style={preaction={transform canvas={shift={(0.5pt,-0.5pt)}}, draw=gray, opacity=0.

In [20]:
def get_adder(n, sumando_1, sumando_2):
    eng = MainEngine()

    a = eng.allocate_qureg(n)
    b = eng.allocate_qureg(n+1)
    c = eng.allocate_qureg(n)

    for i in range(n):
        if sumando_1[i] == "1":
            NOT | (a[n - (i+1)])
    for i in range(n):
        if sumando_2[i] == "1":
            NOT | (b[n - (i+1)])

    for i in range(n-1):
        Toffoli | (a[i], b[i], c[i+1])
        CNOT | (a[i], b[i])
        Toffoli | (a[i], b[i], c[i+1])

    Toffoli | (a[n-1], b[n-1], b[n])
    CNOT | (a[n-1], b[n-1])
    Toffoli | (a[n-1], b[n-1], b[n])  

    CNOT | (c[n-1], b[n-1])

    for i in range(n-1):
        Toffoli | (c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        CNOT | (a[(n-2)-i], b[(n-2)-i])
        Toffoli | (a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])

        CNOT | (c[(n-2)-i], b[(n-2)-i])
        CNOT | (a[(n-2)-i], b[(n-2)-i])

    All(Measure) | b

    return eng

def simulate(circuit):
    circuit.flush()

    result = []
    for i in range(n):
        result.append(int(b[i]))
    
    return result
    

In [21]:
import timeit

times=[]
for i in range(1,9):
    s = f"""\
from __main__ import get_adder
from __main__ import simulate

circuit = get_adder({i},"{"0"*i}1","{"0"*i}1") 
"""
    times.append(timeit.timeit('simulate(circuit)', setup=s, number=100))
    print(i,times[-1])

print(times)

1 0.0022730000000592554
2 0.0020450000010896474
3 0.0022564000009879237
4 0.0018098000000463799
5 0.0021084999989398057
6 0.001689799999439856
7 0.003349099999468308
8 0.0019701999990502372
[0.0022730000000592554, 0.0020450000010896474, 0.0022564000009879237, 0.0018098000000463799, 0.0021084999989398057, 0.001689799999439856, 0.003349099999468308, 0.0019701999990502372]
